In [1]:
#libarary
import sys
from pyspark.sql import SparkSession, functions, types
import pandas as pd
import exifread
import sys
import numpy as np 

In [14]:
spark = SparkSession.builder.appName('reddit averages').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

In [15]:
assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
assert spark.version >= '2.3' # make sure we have Spark 2.3+

In [16]:
amenity_schema = types.StructType([
    types.StructField('lat', types.DoubleType(), nullable=False),
    types.StructField('lon', types.DoubleType(), nullable=False),
    types.StructField('timestamp', types.TimestampType(), nullable=False),
    types.StructField('amenity', types.StringType(), nullable=False),
    types.StructField('name', types.StringType(), nullable=True),
    types.StructField('tags', types.MapType(types.StringType(), types.StringType()), nullable=False),
])

In [2]:
# make pandas dataframe with metadata of image
def photoToDf(img_path):
    path_name = img_path
    f = open(path_name, 'rb')
    tags = exifread.process_file(f)
    lst = []
    col = []
    for tag in tags.keys():
        if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
#         print ("Key: %s, Value: %s" % (tag, tags[tag]))
#         print (tag, tags[tag])
            col.append(tag)
            lst.append(tags[tag])
    df = pd.DataFrame(lst, index=col)                        
    df = df.transpose()    
    return df

In [33]:
# make pandas dataframe with map data
def getMapData(data_path):
    amenity = spark.read.json(data_path,schema=amenity_schema)
    amenity = amenity.toPandas()
    return amenity

In [36]:
def main(img_path, data_path, output):
    df = photoToDf(img_path)
#     display(df)
    dfMap = getMapData(data_path)
    display(dfMap)

#     df.to_csv(output, sep=",")

In [37]:
if __name__=='__main__':
    #test
    img_path = 'IMG_0029.jpg'
    data_path = "amenities-vancouver.json.gz"
    output = "output"
    
    
#     img_path = sys.argv[1]
#     data_path = sys.argv[2]    
#     output = sys.argv[3]
    main(img_path, data_path, output)

,lat,lon,timestamp,amenity,name,tags
0,49.260812,-123.125736,2020-03-20 18:22:12,cafe,Starbucks,"{'brand:wikidata': 'Q37158', 'official_name': ..."
1,49.260953,-123.125704,2019-08-02 18:11:20,fast_food,Salad Loop,{'opening_hours': 'Mo-Fr 07:00-17:00; Sa 10:00...
2,49.373423,-123.291894,2016-10-10 02:14:29,toilets,None,{}
3,49.249848,-122.959708,2011-09-06 03:52:10,bbq,None,{}
4,49.370898,-123.280448,2015-05-03 00:42:25,place_of_worship,St. Monica's Anglican Church,"{'addr:housenumber': '6404', 'addr:street': 'W..."
...,...,...,...,...,...,...
17713,49.278424,-122.806704,2013-03-26 23:45:49,cafe,Creekside Coffee,{}
17714,49.278770,-122.797628,2013-03-26 23:45:49,restaurant,Togo Sushi,{'cuisine': 'japanese'}
17715,49.276443,-122.790138,2013-03-26 23:45:50,parking,None,{}
17716,49.282666,-122.826978,2019-09-13 13:56:49,pub,Brown's Social House,"{'brewery': 'Guinness;various', 'addr:housenum..."
